In [ ]:
import numpy as np
import pandas as pd
import csv
import json



# q2



In [138]:
data_file = r"../../InputData/Adult/adult.data"
query_file_prefix = r"../../InputData/Adult/query"
constraint_file_prefix = r"../../InputData/Adult/constraint"


In [139]:
data = pd.read_csv(data_file, index_col=False)



In [140]:

data['workclass'].unique()



array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [141]:
data['relationship'].unique()


array(['Not-in-family', 'Husband', 'Wife', 'Own-child', 'Unmarried',
       'Other-relative'], dtype=object)

In [142]:
q = 2
c = 1
query_file = query_file_prefix + str(q) + ".json"
constraint_file = constraint_file_prefix + str(c) + ".json"

In [143]:
# read in query and constraint

with open(query_file) as f:
    query_info = json.load(f)

numeric_attributes = []
categorical_attributes = {}
selection_numeric_attributes = {}
selection_categorical_attributes = {}
if 'selection_numeric_attributes' in query_info:
    selection_numeric_attributes = query_info['selection_numeric_attributes']
    numeric_attributes = list(selection_numeric_attributes.keys())
if 'selection_categorical_attributes' in query_info:
    selection_categorical_attributes = query_info['selection_categorical_attributes']
    categorical_attributes = query_info['categorical_attributes']
selected_attributes = numeric_attributes + [x for x in categorical_attributes]
print("selected_attributes", selected_attributes)

with open(constraint_file) as f:
    constraint_info = json.load(f)

sensitive_attributes = constraint_info['all_sensitive_attributes']
fairness_constraints = constraint_info['fairness_constraints']

pd.set_option('display.float_format', '{:.2f}'.format)



selected_attributes ['hours-per-week', 'capital-loss', 'occupation', 'relationship']


In [144]:
print(len(data))
data1 = data[data['hours-per-week'] <= 50]
print(len(data1))
data2 = data1[data1['capital-loss'] >= 1000]
print(len(data2))
data3 = data2[(data2["occupation"] == "Exec-managerial") | (data2["occupation"] =="Prof-specialty") | (data2["occupation"] =="Protective-serv")]
print(len(data3))
data4 = data3[(data3["relationship"] == "Husband") | (data3["relationship"] =="Wife") | (data3["relationship"] =="Own-child")]
print(len(data4))


32561
28918
1230
471
350


In [145]:
data4_female = data4[data4['sex']=='Female']
print(len(data4_female))

data4_male_white = data4[(data4['sex'] == 'Male') & (data4['race'] == 'White')]
print(len(data4_male_white))



50
277



# q1




In [148]:

data_file = r"../../InputData/Adult/adult.data"
query_file_prefix = r"../../InputData/Adult/query"
constraint_file_prefix = r"../../InputData/Adult/constraint"

data = pd.read_csv(data_file, index_col=False)





In [149]:


print(len(data))
data1 = data[data['education-num'] >= 14]
print(len(data1))
data2 = data1[data1['hours-per-week'] >= 21]
print(len(data2))
data3 = data2[data2['capital-gain'] >= 5501]
print(len(data3))


32561
2712
2567
401


In [151]:

data3_female = data3[data3['sex']=='Female']
print(len(data3_female))

data3_male_white = data3[(data3['sex'] == 'Male') & (data3['race'] == 'White')]
print(len(data3_male_white))


# data3 = data2[(data2["occupation"] == "Exec-managerial") | (data2["occupation"] =="Prof-specialty") | (data2["occupation"] =="Protective-serv")]
# print(len(data3))
# data4 = data3[(data3["relationship"] == "Husband") | (data3["relationship"] =="Wife") | (data3["relationship"] =="Own-child")]
# print(len(data4))


70
304


In [126]:

# data satisfying query
def test_satisfying_rows(row):
    terms = row[selected_attributes].to_dict()
    for k in terms:
        if k in selection_numeric_attributes:
            if not eval(
                    str(terms[k]) + selection_numeric_attributes[k][0] + str(selection_numeric_attributes[k][1])):
                return 0
        else:
            if terms[k] not in selection_categorical_attributes[k]:
                return 0
    return 1


data['protected_greater_than'] = 0
data['protected_smaller_than'] = 0
data['satisfy'] = 0

data['satisfy'] = data.apply(test_satisfying_rows, axis=1)

# all_relevant_attributes = sensitive_attributes + selected_attributes + \
#                           ['protected_greater_than', 'protected_smaller_than', 'satisfy']
# data = data[all_relevant_attributes]
# data = data.groupby(all_relevant_attributes, dropna=False, sort=False).size().reset_index(name='occurrence')
all_attributes = data.columns.tolist()
data = data.groupby(all_attributes, dropna=False, sort=False).size().reset_index(name='occurrence')


In [127]:
data[:4]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Unnamed: 14,protected_greater_than,protected_smaller_than,satisfy,occurrence
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,0,0,0,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0,0,0,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0,0,0,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0,0,0,1


In [128]:

data_satisfy = data[data['satisfy']==1]
print(len(data_satisfy))
data_satisfy[:4]



350


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Unnamed: 14,protected_greater_than,protected_smaller_than,satisfy,occurrence
32,45,Private,386940,Bachelors,13,Divorced,Exec-managerial,Own-child,White,Male,0,1408,40,United-States,<=50K,0,0,1,1
327,43,Private,187728,Some-college,10,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,1887,50,United-States,>50K,0,0,1,1
468,49,Private,192776,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,45,United-States,>50K,0,0,1,1
597,42,Self-emp-not-inc,214242,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,1902,50,United-States,>50K,0,0,1,1


In [129]:


data_female = data_satisfy[data_satisfy['sex'] == 'Female']

data_female['occurrence'].sum()


50

In [130]:


data_male_white = data_satisfy[(data_satisfy['sex'] == 'Male') & (data_satisfy['race'] == 'White')]

data_male_white['occurrence'].sum()


277